In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time
import random
from sqlalchemy import create_engine
from configparser import ConfigParser

In [ ]:
def scrapping():
    #[WIP] Ajustar las listas (all_articles y all_divs) para buscar distintos articulos dentro de un mismo tipo de articulos
    #[WIP] No se recogen los procesadores!
    driver = webdriver.Chrome()
    
    all_articles = ['placas-base', 'procesadores', 'discos-duros', 'discos-duros-ssd', 'tarjetas-graficas', 
             'memorias-ram', 'grabadoras-dvd-blu-ray', 'multilectores', 'tarjetas-sonido', 'torres', 
             'ventiladores', 'fuentes-alimentacion', 'modding', 'capturadoras', 
             'cables-internos-de-pc', 'conectividad']
    
    all_divs = ['Placas Base', 'Procesadores', 'Discos Duros', 'discos duros ssd', 'Tarjetas Gráficas', 'memorias ram', 'Grabadores DVD/Blu Ray', 'Multilectores', 'Tarjetas Sonido',
                'Torres ATX', 'Barebones', 'Phanteks', 'Accesorios Torres', '', 'Refrigeración Líquida', 'Pasta térmica', 'Ventiladores CPU', 'Fuentes Alimentación', 'Modding', 
                'Capturadoras', 'Cables internos', 'Conectividad'] 
    
    global articles_list, categories_list, prices_list, stock_list
    articles_list = []
    categories_list = []
    prices_list = []
    stock_list = []
    
    cnt = 0
    for i in all_articles:
        cnt +=1
        print(i)
        driver.get(url="https://www.pccomponentes.com/"+i)
        driver.maximize_window() 
        
        while True:
            try:
                #Accept Cookies
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-block btn-primary  btn-lg m-t-1 accept-cookie']"))).click() 
                print("Cookies aceptadas")
                break
            except TimeoutException:
                break
        
        while True:
            try:
                #Accept Cookies
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-secondary btn-block btn-lg btn-more']"))).click() 
                print("Ver más...")
                break
            except TimeoutException:
                break
                
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        #Randoms mouse movements
        action = ActionChains(driver)
        
        total_articles = driver.find_element_by_xpath("//strong[@id='totalArticles']").text
        total_articles = int(total_articles)
        print(total_articles)
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
            # Wait to load page
            time.sleep(1.5)
            
        
            action.move_by_offset(random.randint(0,10),random.randint(0,10));
            action.perform();
            time.sleep(0.1)
            
            #Scroll top
            driver.execute_script("window.scrollTo(0, 0);")
                                  
            # Wait to load page
            time.sleep(1.5)
        
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # If heights are the same it will exit the function
                break
            last_height = new_height
            
        
            
        
        div = driver.find_elements_by_xpath("//article[@data-category="+"'"+all_divs[cnt-1]+"'"+']')
              
        for i in div:
            articles_list.append(i.get_attribute('data-name'))
            categories_list.append(i.get_attribute('data-category'))
            prices_list.append(i.get_attribute('data-price'))
            stock_list.append(i.get_attribute('data-stock-web'))
            
            
        if len(articles_list) == total_articles: #Check scrapped all the articles
            print('¡Todos los articulos scrapeados!')
            pass
        else:
            print('¡¡¡¡¡¡¡No se han recogido todos los articulos!!!!!!!')
            scrapping()
            
        print('Numero total de articulos: '+str(total_articles))
        
    return articles_list, categories_list, prices_list, stock_list

scrapping()

In [ ]:
all_articles = ['placas-base', 'procesadores', 'discos-duros', 'discos-duros-ssd', 'tarjetas-graficas', 
             'memorias-ram', 'grabadoras-dvd-blu-ray', 'multilectores', 'tarjetas-sonido', 'torres', 
             'ventiladores', 'fuentes-alimentacion', 'modding', 'capturadoras', 
             'cables-internos-de-pc', 'conectividad']
for i in all_articles:
    print (i[1][0])

In [ ]:
a = 'ASD'
print("HOLA"+'['+"'"+a+"'"+']')

In [ ]:
lista = ['hola', 'mesa', 'adios']

In [ ]:
for i in lista:
    print(i)

In [ ]:
def convert_dataframe(articles_list, categories_list, prices_list, stock_list):
    
    global data
    
    data_tuples = list(zip(articles_list, categories_list, prices_list, stock_list))
    data = pd.DataFrame(data_tuples, columns=['Articulo','Categoria', 'Precio', 'Stock'])
    
    
    return data

In [ ]:
convert_dataframe(articles_list, _categories_list, prices_list, stock_list)

In [ ]:
config = ConfigParser()
config.read('/home/jacobo/Config_Trash/database.ini')
user = config['MYSQL']['user']
password = config['MYSQL']['password']
ip = config['MYSQL']['ip']
port = config['MYSQL']['port']
database = config['MYSQL']['database']

In [ ]:
con = create_engine('mysql://'+user+':'+password+'@'+ip+':'+port+'/'+database)

In [ ]:
data.to_sql(con=con, name='TABLA_DE_PRUEBA', if_exists='replace')